In [ ]:
import glob, os
import pandas as pd

In [ ]:
%run -i '../scripts/video_asset_processor.py' 

In [ ]:
metrics_dict = {}
originals_path = '../data/{}/'
for original_asset in glob.iglob(originals_path.format(7) + '**', recursive=False):
    if os.path.isfile(original_asset): # filter dirs
        try:
            start_time = time.time()
            renditions_list = []
            
            #renditions_list.append(original_asset)
            for i in range(3,7):
                rendition_folder = originals_path.format(i)
                renditions_list.append(rendition_folder + os.path.basename(original_asset))
            asset_processor = video_asset_processor(original_asset, renditions_list)
            asset_processor.display = False
            asset_metrics_dict = asset_processor.process()

            dict_of_df = {k: pd.DataFrame(v) for k,v in asset_metrics_dict.items()}
            df = pd.concat(dict_of_df, axis=1).transpose()
            metrics = ['cosine', 'euclidean', 'hamming', 'psnr', 'ssim']
            grouped_df = df.groupby(['dimensions'] + metrics, as_index=False).count()
            renditions_dict = {}
            for rendition in df['dimensions'].unique():
                rendition_dict = {}
                for metric in metrics:
                    rendition_dict[metric] = grouped_df[grouped_df['dimensions']==rendition][metric].mean()
                renditions_dict[rendition] = rendition_dict
            metrics_dict[original_asset] = renditions_dict   
            
            elapsed_time = time.time() - start_time 
            print('Elapsed time:', elapsed_time)
        except:
            print('Failed to process asset:', original_asset)

In [ ]:
dict_of_df = {k: pd.DataFrame(v) for k,v in metrics_dict.items()}
df = pd.concat(dict_of_df, axis=1).transpose().reset_index(inplace=False)
df.to_csv('../output/metrics.csv')
